In [ ]:
import argparse
import gym
import numpy as np
from itertools import count
from collections import namedtuple

from TD3.td3 import TD3

args = {    
    'render': True,
    'log_interval': 1
}

env = gym.make('BipedalWalker-v2')
episodes = 100000
reward_history = []

def main():   
    task = {
        'state_size': 24,
        'action_size': 4,
        'action_high': 1,
        'action_low': -1
    }
    agent = TD3(task)
    sum_reward = 0   
    for i_episode in range(episodes):
        probs_history = [[],[],[],[]]
        running_reward = 0        
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action, noise_coeff = agent.act(state, i_episode)                
            state, reward, done, _ = env.step(action) 
            agent.step(action, reward, state, done, t)
            for k in range(4):
                probs_history[k].append(action[k])
            if args['render']:
                env.render()                   
            running_reward += reward            
            if done:
                break
        
        reward_history.append(running_reward)
        
        if i_episode % args['log_interval'] == 0:
            avg_reward = np.mean(reward_history[-100:])   
            print('Episode {}   Last length: {:5d}   Reward: {:7.2f}   Avg Reward: {:7.2f}   Noise: {:.2f}   \nProbs: {:.4f} {:.4f} {:.4f} {:.4f}'.format(
                i_episode, t, running_reward, avg_reward, noise_coeff, np.mean(probs_history[0]), np.mean(probs_history[1]), np.mean(probs_history[2]), np.mean(probs_history[3])))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break

print("action_space={}".format(env.action_space))
print("obs_space={}".format(env.observation_space))
print("threshold={} \n".format(env.spec.reward_threshold))
main()



Using TensorFlow backend.


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
action_space=Box(4,)
obs_space=Box(24,)
threshold=300 

Episode 0   Last length:   710   Reward: -152.80   Avg Reward: -152.80   Noise: 1.00   
Probs: 0.0622 0.0233 0.3269 0.1225
Episode 1   Last length:    93   Reward: -110.13   Avg Reward: -131.47   Noise: 1.00   
Probs: 0.6036 0.7784 0.5920 0.4002
